# Check the Using Transformers 1. Behind the Pipeline for notes


In [1]:
from transformers import(
    AutoTokenizer,
    TFAutoModel,
    TFAutoModelForSequenceClassification
    ) 
import tensorflow as tf

In [2]:
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [3]:
tokenizer

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased-finetuned-sst-2-english', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [4]:
raw_input = [
    'Hey I am for a test',
    'All the best'
]
inputs = tokenizer(raw_input, padding=True, truncation=True, return_tensors='tf')
print(inputs)

Metal device set to: {'input_ids': <tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[ 101, 4931, 1045, 2572, 2005, 1037, 3231,  102],
       [ 101, 2035, 1996, 2190,  102,    0,    0,    0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 8), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 0, 0, 0]], dtype=int32)>}


2022-01-19 14:20:58.716326: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-19 14:20:58.716925: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [ ]:
# {'input_ids': <tf.Tensor: shape=(2, 8), dtype=int32, numpy=
# array([[ 101, 4931, 1045, 2572, 2005, 1037, 3231,  102],
#        [ 101, 2035, 1996, 2190,  102,    0,    0,    0]], dtype=int32)>, 
# 'attention_mask': <tf.Tensor: shape=(2, 8), dtype=int32, numpy=
# array([[1, 1, 1, 1, 1, 1, 1, 1],
#        [1, 1, 1, 1, 1, 0, 0, 0]], dtype=int32)>}

In [6]:
model = TFAutoModel.from_pretrained(checkpoint)

2022-01-19 14:23:42.958406: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertModel: ['pre_classifier', 'dropout_19', 'classifier']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the tas

In [7]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)

(2, 8, 768)


In [9]:
#dir(outputs)

In [11]:
senti_model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = senti_model(inputs)

Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_38']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
outputs

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-1.4058973,  1.5513633],
       [-4.28962  ,  4.6092067]], dtype=float32)>, hidden_states=None, attentions=None)

In [ ]:
# TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
# array([[-1.4058973,  1.5513633],
#        [-4.28962  ,  4.6092067]], dtype=float32)>, hidden_states=None, attentions=None)

In [13]:
print(outputs.logits)


tf.Tensor(
[[-1.4058973  1.5513633]
 [-4.28962    4.6092067]], shape=(2, 2), dtype=float32)


In [15]:
predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)

tf.Tensor(
[[4.9394473e-02 9.5060551e-01]
 [1.3653041e-04 9.9986351e-01]], shape=(2, 2), dtype=float32)


In [16]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

## Model Inference 

In [3]:
sequences = ["Hello!", "Cool.", "Nice!"]

encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

model = TFAutoModel.from_pretrained("bert-base-cased")
model_inputs = tf.constant(encoded_sequences)

output = model(model_inputs)

Downloading: 100%|██████████| 570/570 [00:00<00:00, 310kB/s]
Downloading: 100%|██████████| 502M/502M [00:23<00:00, 22.6MB/s]
2022-01-19 15:21:01.512029: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-19 15:21:01.512730: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [8]:
print(output.last_hidden_state.shape)

(3, 4, 768)


## Tokenizers

In [2]:
from transformers import AutoTokenizer

In [4]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

Downloading: 100%|██████████| 29.0/29.0 [00:00<00:00, 13.5kB/s]
Downloading: 100%|██████████| 208k/208k [00:00<00:00, 2.53MB/s]
Downloading: 100%|██████████| 426k/426k [00:00<00:00, 3.43MB/s]


In [5]:
tokenizer("An intellectual unable to make his core teachings heard")

{'input_ids': [101, 1760, 8066, 3372, 1106, 1294, 1117, 4160, 12815, 1767, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [6]:
tokenizer.tokenize("An intellectual unable to make his core teachings heard")

['An',
 'intellectual',
 'unable',
 'to',
 'make',
 'his',
 'core',
 'teachings',
 'heard']

In [7]:
tokenizer.convert_tokens_to_ids("An intellectual unable to make his core teachings heard")

100

In [12]:
tokenizer.decode([20, 30, 57, 7993])

'[unused20] [unused30] [unused57] Using'

## Handling multiple sequences

In [13]:
tokenizer.pad_token_id

0